In [1]:
from database.DataBaseModels import *
from source.ArticleParser import ArticleParser

In [5]:
@db_session
def commit_in_db(db_session, article_data):
    article_entity = db_session.Article.get(article_id=article_data['article_id'])
    
    if not article_entity:
        
        venue_entity = db_session.Venue.get(name = str(article_data['venue']['name']))
        if not venue_entity:
            venue_entity = db_session.Venue(name = str(article_data['venue']['name']), url = article_data['venue']['url'])
            db_session.commit()
        
        article_entity = db.Article(article_id = article_data['article_id'],
                  title = article_data['title'],
                  abstract = article_data['abstract'],
                  year = article_data['year'],
                  venue_id = venue_entity.venue_id)
        db_session.commit()
        
        for aa in article_data['authors_and_affiliations']:
            
            author_entity = db_session.Author.get(name = aa['name'])
            if not author_entity:
                author_entity = db_session.Author(name = aa['name'], url = aa['url'])
                db_session.commit()
            
            affiliation_entity = db_session.Affiliation.get(url = aa['affiliation']['url'])
            if not affiliation_entity:
                affiliation_entity = db_session.Affiliation(name = aa['affiliation']['name'],
                                                                url = aa['affiliation']['url'])
                db_session.commit()
            
            db_session.AuthorArticle(author_id = author_entity.author_id,
                                     article_id = article_entity.article_id,
                                    affiliation_id = affiliation_entity.affiliation_id)
        
        db_session.commit()
        
        for citation in article_data['cited_by']:
            db_session.Citation(article_id = article_entity.article_id,
                               cited_by = int(citation))
            
        db_session.commit()
    else:
        print('Article with such id {0} already exists'.format(article_entity.article_id))

In [6]:
total = len(db.Citation.select())
counter = 0
success_counter = 0

citations_list = db.Citation.select()

for citation in citations_list[:1]:
    try:
        counter += 1
        print('Progress: {0}\{1}'.format(counter, total))
        article_parser = ArticleParser()
        article_data = article_parser.parse(citation.cited_by)
        commit_in_db(db_session=db, article_data=article_data)
        success_counter += 1
        print('\n\n')
    except Exception as e:
        print(str(e) + '\n\n')
            
print('{0} from {1} articles were parsed successfully'.format(success_counter, total))

Progress: 1\6049
Start parsing article with id: 3098231
Initializing...
Getting authors...
Getting abstract...
Getting venue...
Get citations...
Successfully!!! ^_^
'authors'


0 from 6049 articles were parsed successfully
